In [1]:
import pandas as pd
import FinanceDataReader as fdr
import yfinance as yf
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

import yfinance as yf
import pandas as pd
import numpy as np
from yahoo_fin import stock_info as si
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.preprocessing import StandardScaler
from utils.prepro import preprocess_stock

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

### Get dataset list

1. We collect the dataset from FDR package 
2. We drop the non stock such as "fund"

In [2]:
kospi_symbol = fdr.StockListing('KOSPI')
nasdaq_symbol = fdr.StockListing('NASDAQ')
nyse_symbol = fdr.StockListing('NYSE')

sse_stocks = fdr.StockListing('SSE')   #상해거래소
HKEX_stocks = fdr.StockListing('HKEX') #홍콩거래소
TSE_stocks = fdr.StockListing('TSE')   #도쿄거래소
SZSE_stocks = fdr.StockListing('SZSE') #산전거래소

nasdaq_symbol["exchange"] = "NASDAQ"
sse_stocks["exchange"] = "SSE"
TSE_stocks["exchange"] = "TSE"
kospi_symbol["exchange"] = "KOSPI"
HKEX_stocks["exchange"] = "HKEX"
SZSE_stocks["exchange"] = "SZSE"
nyse_symbol["exchange"] = "NYES"

100% 3802/3802 [00:06<00:00, 567.58it/s]
100% 2821/2821 [00:04<00:00, 573.18it/s]
100% 1542/1542 [00:01<00:00, 1036.70it/s]
100% 2677/2677 [00:02<00:00, 971.19it/s] 
100% 4013/4013 [00:04<00:00, 927.10it/s] 
100% 1808/1808 [00:02<00:00, 855.85it/s]


In [3]:
asdaq_symbol = nasdaq_symbol[~nasdaq_symbol["Industry"].str.contains("펀드", na=False)]
se_stocks = sse_stocks[~sse_stocks["Industry"].str.contains("펀드", na=False)]
SE_stocks = TSE_stocks[~TSE_stocks["Industry"].str.contains("펀드", na=False)]
KEX_stocks = HKEX_stocks[~HKEX_stocks["Industry"].str.contains("펀드", na=False)]
ZSE_stocks = SZSE_stocks[~SZSE_stocks["Industry"].str.contains("펀드", na=False)]
yse_symbol = nyse_symbol[~nyse_symbol["Industry"].str.contains("펀드", na=False)]

In [4]:
kospi_symbol["Symbol"] = kospi_symbol["Code"] 

In [5]:
stock_list = pd.concat([nasdaq_symbol, nyse_symbol]).dropna().reset_index(drop = True)
stock_list = stock_list.drop("Name", axis = 1)
stock_list = stock_list.drop_duplicates().reset_index(drop = True)

### Get dataframe

In [6]:
# Notice that : 2023년 1월 1일 부터 2023년 8월 31일까지 발행된 정보(2023년 8월 31일 기준)

def fetch_data(symbol, exchange, industry_code):
    try:
        if exchange == "TSE":
            symbol = symbol + ".T"
        elif exchange == "SZSE":
            symbol = symbol + ".SZ"
        elif exchange == "SSE":
            symbol = symbol + ".SS"
        elif exchange == "KOSPI":
            symbol = symbol + ".KS"    
        
        ticker = yf.Ticker(symbol)
        df = ticker.history(start="2018-01-01",end = "2024-03-28", interval="1d").reset_index()[["Date", "Open","High","Low","Close","Volume"]]   
        df = preprocess_stock(df)
        df["Stock_"] = symbol
        df["IndustryCode_"] = industry_code
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
    return df

def get_all_data(stock_list):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_data, symbol, exchange, industry_code) for symbol, exchange, industry_code in zip(stock_list["Symbol"], stock_list["exchange"], stock_list["IndustryCode"])]
        results = [future.result() for future in as_completed(futures)]

    data = pd.concat([result for result in results if result is not None])

    return data

In [7]:
nasdaq_data = get_all_data(stock_list)
TSE_data = get_all_data(TSE_stocks)
SZSE_data = get_all_data(SZSE_stocks)
sse_data = get_all_data(sse_stocks)

FAAS: Data doesn't exist for startDate = 1514782800, endDate = 1711598400
BOLD: Data doesn't exist for startDate = 1514782800, endDate = 1711598400
Got error from yahoo api for ticker GWACU, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- GWACU: No timezone found, symbol may be delisted
ZBAO: Data doesn't exist for startDate = 1514782800, endDate = 1711598400
MAMO: Data doesn't exist for startDate = 1514782800, endDate = 1711598400
UBXG: Data doesn't exist for startDate = 1514782800, endDate = 1711598400
ZPTA: Data doesn't exist for startDate = 1514782800, endDate = 1711598400
NCAC: No data found, symbol may be delisted
NIVF: Period '1mo' is invalid, must be one of ['1d', '5d']
Got error from yahoo api for ticker GIPRU, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- GIPRU: No timezone found, symbol may be delisted
CLVSQ: No data found, symbol may be delisted
Got error from yahoo api for ticker MAXNV, Error: 

### Data Cleaning

In [8]:
columns = ["Open00",'Open06', 'Open12', 'Open18', 'Open24', 'Open30',
              "High00",'High06', 'High12', 'High18', 'High24', 'High30',
               "Low00",'Low06', 'Low12','Low18', 'Low24', 'Low30',
              "Close00",'Close06', 'Close12', 'Close18', 'Close24', 'Close30',
               "Volume00",'Volume06', 'Volume12', 'Volume18', 'Volume24', 'Volume30']

def get_check_data(data):
    data_length = data["Stock_"].value_counts().reset_index()
    data = data[data["Stock_"].isin(data_length[data_length["Stock_"] >= 1000].iloc[:, 0].values)].reset_index(drop = True)
    data["Date"] = data["Date"].astype(str)
    date_ = [data["Date"][i][:10] for i in range(data.shape[0])]
    data["Date"] = date_
    return data

In [9]:
nasdaq_data2 = get_check_data(nasdaq_data)
TSE_data2    = get_check_data(TSE_data)
SZSE_data2   = get_check_data(SZSE_data)
sse_data2    = get_check_data(sse_data)

In [10]:
print("nasdaq data shape: ", nasdaq_data2.shape)
print("tse data shape: ", TSE_data2.shape)
print("szse data shape: ", SZSE_data2.shape)
print("sse data shape: ", sse_data2.shape)

nasdaq data shape:  (6311889, 38)
tse data shape:  (4972029, 38)
szse data shape:  (2214980, 38)
sse data shape:  (1666886, 38)


In [11]:
# Encoding 
# Date (We will update)
# Sector 

In [12]:
def date_encoder_fun(data):
    date_encoder = LabelEncoder()
    date_encoder.fit(data["Date"].unique())
    data["Date_index"] = date_encoder.transform(data["Date"])
    return data

In [13]:
TSE_date    = date_encoder_fun(TSE_data2)
NASDAQ_date = date_encoder_fun(nasdaq_data2)
SZSE_date   = date_encoder_fun(SZSE_data2)
SSE_date    = date_encoder_fun(sse_data2)

In [14]:
def sector_encoder_fun(data):
    date_encoder = LabelEncoder()
    date_encoder.fit(data["IndustryCode_"].unique())
    data["Sector"] = date_encoder.transform(data["IndustryCode_"])
    return data

In [15]:
TSE     =   sector_encoder_fun(TSE_date)
NASDAQ = sector_encoder_fun(NASDAQ_date)
SZSE    = sector_encoder_fun(SZSE_date)
SSE    = sector_encoder_fun(SSE_date)

In [16]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] <= "2023-01-01"].reset_index(drop = True)
    test = data[data["Date"] > "2023-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_{}.csv".format(name), index = False)
    else:
        return train, test

In [ ]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")

#### data

In [18]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] < "2022-01-01"].reset_index(drop = True)
    test = data[data["Date"] >= "2022-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_2022_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_2022_{}.csv".format(name), index = False)
    else:
        return train, test

In [19]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")

In [20]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] < "2021-01-01"].reset_index(drop = True)
    test = data[data["Date"] >= "2021-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_2021_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_2021_{}.csv".format(name), index = False)
    else:
        return train, test

In [21]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")

In [22]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] < "2020-01-01"].reset_index(drop = True)
    test = data[data["Date"] >= "2020-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_2020_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_2020_{}.csv".format(name), index = False)
    else:
        return train, test

In [23]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")

In [24]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] < "2019-01-01"].reset_index(drop = True)
    test = data[data["Date"] >= "2019-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_2019_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_2019_{}.csv".format(name), index = False)
    else:
        return train, test

In [ ]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")

In [ ]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] < "2018-01-01"].reset_index(drop = True)
    test = data[data["Date"] >= "2018-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_2018_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_2018_{}.csv".format(name), index = False)
    else:
        return train, test

In [ ]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")

In [ ]:
def get_train_test_split(data, is_save = False, is_norm = True, name = None):
    
    train = data[data["Date"] < "2023-01-01"].reset_index(drop = True)
    test = data[data["Date"] >= "2023-01-01"].reset_index(drop = True)
    
    if is_norm == True:
        scaler = StandardScaler()
        normalized_column = scaler.fit_transform(train[columns])
        train[columns] = normalized_column
        test[columns] = scaler.transform(test[columns])
    else:
        pass
    
    if is_save:
        train.to_csv("./data_ex_fund/train_2023_{}.csv".format(name), index = False)
        test.to_csv("./data_ex_fund/test_2023_{}.csv".format(name), index = False)
    else:
        return train, test

In [ ]:
get_train_test_split(NASDAQ, is_save = True, is_norm = True,name = "nasdaq")
get_train_test_split(TSE, is_save = True, is_norm = True, name = "tse")
get_train_test_split(SZSE, is_save = True,is_norm = True,name =  "szse")
get_train_test_split(SSE, is_save = True, is_norm = True, name = "sse")